In [267]:
#load libraries
from pystackreg import StackReg
import numpy as np
import pandas as pd
import os
import tifffile as tiff
import numpy as np
from skimage.transform import rescale
import pystackreg
from pystackreg import StackReg
import glob
import gc
import os
import sys
import tifffile as tiff
from sys import getsizeof
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import statistics as st
import imagecodecs
from matplotlib.colors import ListedColormap
from skimage import measure, io

In [275]:
#load functions
def get_transformation_mat(ref, mov):
    sr = StackReg(StackReg.RIGID_BODY)
    transformation_matrix = sr.register(ref, mov)
    return transformation_matrix

def get_trans_mat_for_visualization(ref, mov): 
    sr = StackReg(StackReg.RIGID_BODY)
    sr.register(ref, mov)
    #print('Registration matrix acquired')
    return(sr)

def transform_coordinates(transformation_matrix, x, y):
    # Create an array of your coordinates, including a row of ones for homogeneous coordinates
    coords = np.vstack([x, y, np.ones_like(x)])
    
    # Apply the transformation matrix to the coordinates
    transformed_coords = np.dot(transformation_matrix, coords)
    
    # Extract the transformed x and y coordinates
    x_transformed = transformed_coords[0, :]
    y_transformed = transformed_coords[1, :]
    
    return x_transformed, y_transformed

def compare_image_pair_sizes(epithelial_img, immune_img):
    try:
        # Read the epithelial and immune TIFF images
        #epithelial_img = tiff.imread(epithelial_image_path)
        #immune_img = tiff.imread(immune_image_path)
        
        # Initialize variables for comparison results
        height_difference = abs(epithelial_img.shape[0] - immune_img.shape[0])
        width_difference = abs(epithelial_img.shape[1] - immune_img.shape[1])
        
        # Determine which image has the smaller dimensions
        smaller_height_img = 'epithelial' if epithelial_img.shape[0] < immune_img.shape[0] else 'immune'
        smaller_width_img = 'epithelial' if epithelial_img.shape[1] < immune_img.shape[1] else 'immune'
        
        # Prepare results
        #image_pair = (os.path.basename(epithelial_img), os.path.basename(immune_img))
    
    except (FileNotFoundError, OSError) as e:
        # Handle errors if the images are not found or cannot be read
        image_pair = (None, None)
        smaller_height_img = None
        smaller_width_img = None
        height_difference = None
        width_difference = None

    return smaller_height_img, height_difference, smaller_width_img, width_difference

def find_and_read_csv(folder_path, description):
    # Create the file name by appending '.csv' to the description
    file_name = f"{description}.csv"
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the file exists and read it into a DataFrame
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        return df
    else:
        return None
    
def output_overlap_image(ref, mov, sr, overlay_image_filename): 
    blobs_aligned = sr.transform(mov)
    
    # Create the subplots
    f, ax = plt.subplots(1, 3, figsize=(20, 9))

    # Assuming 'ref' and 'mov' are your images
    ref = adjust_brightness(ref, gamma=6)  # Adjust the gamma for brightness
    blob_aligned_bright = adjust_brightness(blobs_aligned, gamma=6)  # Adjust the brightness for the aligned image

    # Create the composite image
    after_reg = composite_images([ref, blob_aligned_bright])

    # Plot the images in the subplots
    ax[0].imshow(ref, cmap='gray')
    ax[0].set_title('reference image')
    ax[0].axis('off')

    ax[1].imshow(mov, cmap='gray')
    ax[1].set_title('shifted image')
    ax[1].axis('off')

    ax[2].imshow(after_reg)
    ax[2].set_title('overlay (reference=red, shifted=green)')
    ax[2].axis('off')

    # Save the figure as a PNG
    plt.savefig(overlay_image_filename, bbox_inches='tight')  # Use bbox_inches to avoid extra white space
    plt.close() 

In [ ]:
#make a mini-loop, applied to subset_caf_df_mov, AND TO A SUBSET OF EP_DF.
#if subset_caf_df_mov['filename'] contains any of keywords = ["PDAC1", "PDAC2", "PDAC3", "PDAC5", "PDAC8"], 
#then rotate it 180* making a new column called y_rot with immune_img_og.shape[0] minus y, and x_rot immune_img_og[1] minus x. if there is no keyword, just replaced y_rot, x_rot with y, x
#if subset_ep_df_mov['filename'] contains any of the keywords ["PDAC2", "PDAC3", "PDAC8"]
#then rotate it 180*, making a new column called y_rot with epithelial_img_og.shape[0] minus y, and x_rot epithelial_img_og[1] minus x. if there is no keyword, just replaced y_rot, x_rot with y, x      
def rotate_coordinates(df, keywords, img_shape):
    """
    Rotate coordinates by 180 degrees if the description contains any of the keywords.
    """
    # Determine which columns to use for coordinates
    if 'x' in df.columns and 'y' in df.columns:
        x_col, y_col = 'x', 'y'
    elif 'x_slide' in df.columns and 'y_slide' in df.columns:
        x_col, y_col = 'x_slide', 'y_slide'
    else:
        raise ValueError("Neither 'x' and 'y' nor 'x_slide' and 'y_slide' columns found in the DataFrame.")
    
    # Create new columns for rotated coordinates
    df['y_rot'] = df[y_col]
    df['x_rot'] = df[x_col]
    
    # Apply rotation if description contains any of the keywords
    for keyword in keywords:
        mask = df['description'].str.contains(keyword, na=False)
        if mask.any():
            df.loc[mask, 'y_rot'] = img_shape[0] - df.loc[mask, y_col]
            df.loc[mask, 'x_rot'] = img_shape[1] - df.loc[mask, x_col]
    
    return df

### Load cross-compartment file mappings

In [132]:
#load sample_csvs that I want to pull descriptions from
#add a 'description' column
CAF_input_folder = "CAF_data/Sample_csvs"
# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(CAF_input_folder) if f.endswith('.csv')]
# Initialize an empty list to hold the DataFrames
dataframes = []
# Loop through the CSV files and read them into DataFrames
for csv_file in csv_files:
    file_path = os.path.join(CAF_input_folder, csv_file)
    df = pd.read_csv(file_path)  # Read the CSV into a DataFrame
    dataframes.append(df)       # Append to the list of DataFrames
# Concatenate all the DataFrames into one
sample_CAF_df = pd.concat(dataframes, ignore_index=True)
#add a description column


#read in relevant CAF df, where I will be transforming coordinates
folder_path = 'CAF_data/CAF_measurement_files'
# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
# Initialize an empty list to store DataFrames
dfs = []
# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Combine all DataFrames into a single DataFrame
caf_df_mov = pd.concat(dfs, ignore_index=True)

#separate caf_df_mov into different folders by sample, so that they can be loaded in faster

def format_filename(sample):
    # Split the sample string into parts
    parts = sample.split('_')
    
    # Extract the base part and suffix
    base = parts[0]
    suffix = parts[1]
    
    # Format the filename
    filename = f"{base}_s0_a{suffix}_ac_full_Simple Segmentation.tiff"
    
    return filename

# Create the filename column
caf_df_mov['filename'] = caf_df_mov['sample'].apply(format_filename)

#add a description column
# Create a mapping from 'SampleName' to 'Sample'
sample_mapping = sample_CAF_df.set_index('SampleName')['Sample'].to_dict()

# Add 'description' column to caf_df_mov
caf_df_mov['description'] = caf_df_mov['sample'].map(sample_mapping)

In [133]:
# Path to the folder containing the CSV files
ep_input_folder = "IMC_Data_Ferris/Sample_csvs"

# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(ep_input_folder) if f.endswith('.csv')]

# Initialize an empty list to hold the DataFrames
dataframes = []

# Loop through the CSV files and read them into DataFrames
for csv_file in csv_files:
    file_path = os.path.join(ep_input_folder, csv_file)
    df = pd.read_csv(file_path)  # Read the CSV into a DataFrame
    dataframes.append(df)       # Append to the list of DataFrames

# Concatenate all the DataFrames into one
ep_df = pd.concat(dataframes, ignore_index=True)

In [134]:
immune_input_folder = "IMC_Data_Noor/Sample_csvs"

# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(immune_input_folder) if f.endswith('.csv')]

# Initialize an empty list to hold the DataFrames
dataframes = []

# Loop through the CSV files and read them into DataFrames
for csv_file in csv_files:
    file_path = os.path.join(immune_input_folder, csv_file)
    df = pd.read_csv(file_path)  # Read the CSV into a DataFrame
    dataframes.append(df)       # Append to the list of DataFrames

# Concatenate all the DataFrames into one
immune_df = pd.concat(dataframes, ignore_index=True)

#Load whole immune dataset too!
immune_df_mov = pd.read_csv('adata_PDAC_lymphoid_and_myeloid_annotations_AUG2024.csv')

#2023_PDAC8_TMA_Immune_Noor_120 and makes it like 2023_PDAC1_TMA_Immune_Noor_s0_a120_ac_full_Simple_Segmentation.tiff

def format_filename(sample):
    # Split the sample string into parts
    parts = sample.split('_')
    
    # Extract the base part and suffix
    base0 = parts[0]
    base1 = parts[1]
    base2 = parts[2]
    base3 = parts[3]
    base4 = parts[4]
    suffix = parts[5]
    
    # Format the filename
    filename = f"{base0}_{base1}_{base2}_{base3}_{base4}_s0_a{suffix}_ac_full_Simple_Segmentation.tiff"
    
    return filename

#separate caf_df_mov into different folders by sample, so that they can be loaded in faster
# Create the filename column
immune_df_mov['filename'] = immune_df_mov['sample'].apply(format_filename)

# Create a mapping from 'SampleName' to 'Sample'
sample_mapping = immune_df.set_index('SampleName')['Sample'].to_dict()

# Add 'description' column to caf_df_mov
immune_df_mov['description'] = immune_df_mov['sample'].map(sample_mapping)

In [272]:
#sr = get_transformation_mat(ep_object_mask, immune_object_mask)

In [274]:
def composite_images(imgs, equalize=False, aggregator=np.mean):

    if equalize:
        imgs = [exposure.equalize_hist(img) for img in imgs]

    imgs = [img / img.max() for img in imgs]

    if len(imgs) < 3:
        imgs += [np.zeros(shape=imgs[0].shape)] * (3-len(imgs))

    imgs = np.dstack(imgs)

    return imgs

def adjust_brightness(image, gamma=1.0):
    # Apply gamma correction to increase brightness
    return np.power(image, gamma)

# Move Immune towards Epithelial


In [277]:
# Merge the DataFrames on the "Sample" column
merged_df = pd.merge(ep_df, immune_df, on='Sample', suffixes=('_ep', '_immune'))

# Create a dictionary mapping "TiffName_x" to "TiffName_y"
tiffname_mapping = dict(zip(merged_df['TiffName_ep'], merged_df['TiffName_immune']))

#make tiffname_mapping compatible with the Simple_Segmentation label
new_suffix = "full_Simple_Segmentation.tiff"
# Modify the keys and values to have the new suffix
tiffname_mapping = {
    key.replace("full.tiff", new_suffix): value.replace("full.tiff", new_suffix)
    for key, value in tiffname_mapping.items()
}

#Folders with matched, rotated, and padded ilastik files:
epithelial_ilastik_folder = "/Users/ferris/Desktop/Serial Section Overlay/000_Data/ilastik_object_masks/epithelial_padded_object_masks"
immune_ilastik_folder = "/Users/ferris/Desktop/Serial Section Overlay/000_Data/ilastik_object_masks/immune_padded_object_masks"

epithelial_ilastik_folder_og = "/Users/ferris/Desktop/Serial Section Overlay/000_Data/ilastik_object_masks/epithelial"
immune_ilastik_folder_og = "/Users/ferris/Desktop/Serial Section Overlay/000_Data/ilastik_object_masks/immune"

ep_keywords = ["PDAC2", "PDAC3", "PDAC8"]
im_keywords = ["not_applicable"]

overlap_folder = f"/Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/plots_immune_rot_pad_trans"

for key, value in tiffname_mapping.items():  # key is ep, value is immune
    try:
        # Take a pair of matching images
        ep_object_mask_path = os.path.join(epithelial_ilastik_folder, key)
        immune_object_mask_path = os.path.join(immune_ilastik_folder, value)
        
        # Take the ORIGINAL pair of matching images
        ep_object_mask_path_og = os.path.join(epithelial_ilastik_folder_og, key)
        immune_object_mask_path_og = os.path.join(immune_ilastik_folder_og, value)


        # Check if files exist before proceeding
        if not os.path.exists(ep_object_mask_path) or not os.path.exists(immune_object_mask_path):
            raise FileNotFoundError(f"File(s) not found: {key}, {value}")

        # Load the images
        ep_object_mask = tiff.imread(ep_object_mask_path)
        immune_object_mask = tiff.imread(immune_object_mask_path)
        
        #load the original images (to be able to measure their shapes)
        epithelial_img_og = tiff.imread(ep_object_mask_path_og)
        immune_img_og = tiff.imread(immune_object_mask_path_og)
        
        epithelial_img_og_shape = epithelial_img_og.shape
        immune_img_og_shape = epithelial_img_og.shape

        # Register them
        transform_matrix = get_transformation_mat(ep_object_mask, immune_object_mask)
        sr = get_trans_mat_for_visualization(ep_object_mask, immune_object_mask)
        
        #output image to make sure transformation makes sense
        overlay_image_filename = os.path.join(overlap_folder, key)
        output_overlap_image(ep_object_mask, immune_object_mask, sr, overlay_image_filename)
       
        #apply registration matrix to dataframe
        #Subset caf_df_mov to the rows where filename equals the value
        subset_immune_df_mov = immune_df_mov[immune_df_mov['filename'] == value].copy()
        
        #read in an epithelial subset that matches
        
        # Extract the first description from caf_df_mov, to be able to find matching epithelial file
        first_description = subset_immune_df_mov['description'].iloc[0]
        # Specify the folder path
        folder_path = '000_Data/transformed_coor/epithelial_original_coor'
        # Find and read the matching CSV file into subset_ep_df_mov
        subset_ep_df_mov = find_and_read_csv(folder_path, first_description)
        
        # Apply rotation to subset_immune- unneccessary, this was used as ground truth
        subset_immune_df_mov = rotate_coordinates(subset_immune_df_mov, im_keywords, immune_img_og_shape)

        # Apply rotation to subset_ep_df_mov
        subset_ep_df_mov = rotate_coordinates(subset_ep_df_mov, ep_keywords, epithelial_img_og_shape)

        #grab information necessary for padding correction
        smaller_height_img, height_difference, smaller_width_img, width_difference = compare_image_pair_sizes(epithelial_img_og, immune_img_og)
        
        #OK GREAT
        
        #now
        #if epithelial is smaller height, add half the height difference to the epithelial y value, and call it rot_pad
        if smaller_height_img == 'epithelial':
            subset_ep_df_mov['y_rot_pad'] = subset_ep_df_mov['y_rot'] + height_difference / 2
            subset_immune_df_mov['y_rot_pad'] = subset_immune_df_mov['y_rot']
        else:
            subset_immune_df_mov['y_rot_pad'] = subset_immune_df_mov['y_rot'] + height_difference / 2
            subset_ep_df_mov['y_rot_pad'] = subset_ep_df_mov['y_rot']
        if smaller_width_img == 'epithelial':
            subset_ep_df_mov['x_rot_pad'] = subset_ep_df_mov['x_rot'] + height_difference / 2
            subset_immune_df_mov['x_rot_pad'] = subset_immune_df_mov['x_rot']
        else:
            subset_immune_df_mov['x_rot_pad'] = subset_immune_df_mov['x_rot'] + height_difference / 2
            subset_ep_df_mov['x_rot_pad'] = subset_ep_df_mov['x_rot']

        # Apply the transformation matrix
        x_transformed, y_transformed = transform_coordinates(transform_matrix, subset_immune_df_mov['x_rot_pad'].values, subset_immune_df_mov['y_rot_pad'].values)

        subset_immune_df_mov['x_rot_pad_trans'] = x_transformed
        subset_immune_df_mov['y_rot_pad_trans'] = y_transformed
        

        x_transformed, y_transformed = transform_coordinates(transform_matrix, subset_ep_df_mov['x_rot_pad'].values, subset_ep_df_mov['y_rot_pad'].values)
        subset_ep_df_mov['x_rot_pad_trans_im'] = x_transformed
        subset_ep_df_mov['y_rot_pad_trans_im'] = y_transformed


        # Get the filename
        
        sample_name = subset_immune_df_mov['description'].iloc[0]
        csv_filename_caf = f"/Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/{sample_name}.csv"
        csv_filename_ep = f"/Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/ep_rot_pad_trans/{sample_name}.csv"
        
        # Write subset_df to CSV
        subset_immune_df_mov.to_csv(csv_filename_caf, index=False)
        subset_ep_df_mov.to_csv(csv_filename_ep, index=False)
        
        print(f"Transformed df has been written to {csv_filename_caf}")

    except FileNotFoundError as e:
        # If a file is not found, print the error message and skip to the next iteration
        print(f"Skipping {key}, {value}: {e}")
        continue

    except Exception as e:
        # Handle any other exceptions to ensure the loop continues
        print(f"An error occurred with {key}, {value}: {e}")
        continue

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector3Row2Column5_Punch2_Case12_ID_1_3924.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector2Row3Column7_Punch1_Case157_ID_GSDM_606.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector3Row3Column4_Punch2_Case19_ID_1_45196.csv
Skipping Ferris_TMA_2022_PDAC2_s0_a8_ac_full_Simple_Segmentation.tiff, 2023_PDAC2_TMA_Immune_Noor_s0_a48_ac_full_Simple_Segmentation.tiff: File(s) not found: Ferris_TMA_2022_PDAC2_s0_a8_ac_full_Simple_Segmentation.tiff, 2023_PDAC2_TMA_Immune_Noor_s0_a48_ac_full_Simple_Segmentation.tiff
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector3Row2Column4_Punch2_Case11_ID_1_3583.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector2Row1Column6_Punch1_Case140_ID_GSDM_2417.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector3Row1Column8_Punch2_Case7_ID_1_109753.csv
Skipping Ferris_TMA_2022_PDAC2_s0_a52_ac_full_Simple_Segmentation.tiff, 2023_PDAC2_TMA_Immune_Noor_s0_a12_ac_full_Simple_Segmentation.tiff: File(s) not found: Ferris_TMA_2022_PDAC2_s0_a52_ac_full_Simple_Segmentation.tiff, 2023_PDAC2_TMA_Immune_Noor_s0_a12_ac_full_Simple_Segmentation.tiff
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector3Row3Column2_Punch2_Case17_

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector3Row3Column8_Punch2_Case23_ID_1_4752.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector2Row1Column4_Punch1_Case138_ID_GSDM_2310.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector1Row3Column7_Punch1_Case125_ID_GSDM_1812.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC2_Sector2Row4Column7_Punch1_Case165_ID_GSDM_992.csv
Skipping Ferris_TMA_2022_PDAC2_s0_a100_ac_full_Simple_Segmentation.tiff, 2023_PDAC2_TMA_Immune_Noor_s0_a104_ac_full_Simple_Segmentation.tiff: File(s) not found: Ferris_TMA_2022_PDAC2_s0_a100_ac_full_Simple_Segmentation.tiff, 2023_PDAC2_TMA_Immune_Noor_s0_a104_ac_full_S

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC7_Sector1Row2Column4_Punch1_Case177_ID_1_48081.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC7_Sector2Row8Column3_Punch3_Case168_ID_1_3385.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC7_Sector1Row3Column8_Punch1_Case191_ID_1_77495.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC7_Sector1Row2Column8_Punch1_Case181_ID_1_56604.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC7_Sector1Row6Column3_Punch1_Case216_ID_GSDM_1472.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC7_Sector1Row9Column7_Punch2_Case185_ID_1_68643.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC7_Sector2Row3Column6_Punch2_Case206_ID_G_17542.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC7_Sector2Row2Column6_Punch2_Case200_ID_1_92732.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC7_Sector2Row7Column2_Punch2_Case226_ID_GSDM_2547.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC7_Sector2Row6Column5_Punch2_Case223_ID_GSDM_2254.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Da

...

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC4_Sector1Row2Column7_Punch3_Case9_ID_1_3430.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC4_Sector1Row4Column5_Punch3_Case23_ID_1_4752.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC4_Sector1Row4Column7_Punch3_Case25_ID_1_48194.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC4_Sector1Row3Column5_Punch3_Case15_ID_1_41902.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC4_Sector3Row4Column5_Punch3_Case86_ID_G_25536.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transform

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC4_Sector4Row2Column6_Punch3_Case102_ID_G_8288.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC4_Sector1Row3Column8_Punch3_Case18_ID_1_44422.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC4_Sector1Row4Column2_Punch3_Case20_ID_1_4567.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC4_Sector2Row4Column3_Punch3_Case52_ID_1_85532.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC4_Sector1Row2Column8_Punch3_Case10_ID_1_35154.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transfo

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector2Row2Column7_Punch1_Case39_ID_1_69943.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector3Row3Column3_Punch1_Case74_ID_G_11920.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector3Row4Column2_Punch1_Case81_ID_G_22428.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector3Row1Column8_Punch1_Case63_ID_1_93605.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector3Row4Column3_Punch1_Case82_ID_G_23381.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transf

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector4Row3Column4_Punch1_Case106_ID_G_9725.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector1Row2Column5_Punch1_Case5_ID_1_104192.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector2Row1Column5_Punch1_Case29_ID_1_54342.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector3Row2Column2_Punch1_Case65_ID_1_94889.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector4Row2Column5_Punch1_Case100_ID_G_6686.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transf

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector3Row3Column7_Punch1_Case78_ID_G_14742.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector3Row1Column2_Punch1_Case57_ID_1_91050.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector4Row3Column6_Punch1_Case108_ID_GSDM_1018.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector3Row3Column8_Punch1_Case79_ID_G_17185.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/immune_rot_pad_trans/PDAC1_Sector3Row1Column6_Punch1_Case61_ID_1_92541.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/tra

#next step:

Apply correction to immune data, THEN, load in the changed epithelial files, and apply another layer of correction based on caf data to most closely mimic how I modified the original images

# Move CAF towards epithelial... aka, make CAF "mov" and epithelial "ref"

In [243]:
#caf_df_mov #reminder that #xmin and ymin are flipped!!!! Labelled wrong!! Don't use them to doublecheck work

In [230]:
#smaller_height_img, height_difference, smaller_width_img, width_difference = compare_image_pair_sizes(epithelial_img_og, immune_img_og)

In [279]:
# First, make sure tiffname mapping is working
# Path to the tiffname_mapping CSV
mapping_csv_path = '/Users/ferris/Desktop/Serial Section Overlay/000_Data/ROI_matching_csvs/CAF_ep_tiffname_mapping.csv'
# Read the CSV into a dictionary
tiffname_mapping = pd.read_csv(mapping_csv_path, index_col=0).squeeze("columns").to_dict()
#make tiffname_mapping compatible with the Simple_Segmentation label
new_suffix = "full_Simple_Segmentation.tiff"
new_suffix_CAF = "full_Simple Segmentation.tiff"
# Modify the keys and values to have the new suffix
tiffname_mapping = {
    key.replace("full.tiff", new_suffix): value.replace("full.tiff", new_suffix_CAF)
    for key, value in tiffname_mapping.items()
}

#Folders with matched, rotated, and padded ilastik files:
epithelial_ilastik_folder = "/Users/ferris/Desktop/Serial Section Overlay/000_Data/ilastik_object_masks/epithelial_padded_object_masks_for_CAF"
CAF_ilastik_folder = "/Users/ferris/Desktop/Serial Section Overlay/000_Data/ilastik_object_masks/CAF_flipped_padded"

#folders with original ilastik files (will only be used for shape calculations!)
epithelial_ilastik_folder_og = "/Users/ferris/Desktop/Serial Section Overlay/000_Data/ilastik_object_masks/epithelial_padded_object_masks"
CAF_ilastik_folder_og = "/Users/ferris/Desktop/Serial Section Overlay/000_Data/ilastik_object_masks/CAF"

#write a function that take

#keywords, for CAF vs epithelial
# Keywords for rotation
caf_keywords = ["PDAC1", "PDAC2", "PDAC3", "PDAC5", "PDAC8"]
ep_keywords = ["not_applicable_anymore"]
#ep_keywords = ["PDAC2", "PDAC3", "PDAC8"] ep was already flipped during the immune/ep matching!!


overlap_folder = f"/Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/plots_caf_rot_pad_trans"


#write loop using custom functions. THIS is the key loop. Here, I want to add 
#first, load both masks, and figure out which image has the smaller axis, and how much it is smaller by

#1. a step that adjusts for rotations, if a keyword from 02 file is present 
#(take x.shape - x, y.shape - y, of the ORIGINAL image)
#2. a step that adjusts for padding. 
#(compare the already loaded original image with the already loaded padded image, 
#and take the smaller one (doing each axis seperately!!!)
#and ADD half the padding value to all x/y coordinates (this will make it centered)
for key, value in tiffname_mapping.items():  # key is ep, value is immune
    try:
        # Take a pair of matching images
        ep_object_mask_path = os.path.join(epithelial_ilastik_folder, key)
        immune_object_mask_path = os.path.join(CAF_ilastik_folder, value)
        
        # Take the ORIGINAL pair of matching images
        ep_object_mask_path_og = os.path.join(epithelial_ilastik_folder_og, key)
        immune_object_mask_path_og = os.path.join(CAF_ilastik_folder_og, value)


        # Check if files exist before proceeding
        if not os.path.exists(ep_object_mask_path) or not os.path.exists(immune_object_mask_path):
            raise FileNotFoundError(f"File(s) not found: {key}, {value}")

        # Load the images
        ep_object_mask = tiff.imread(ep_object_mask_path)
        immune_object_mask = tiff.imread(immune_object_mask_path)
        
        #load the original images (to be able to measure their shapes)
        epithelial_img_og = tiff.imread(ep_object_mask_path_og)
        immune_img_og = tiff.imread(immune_object_mask_path_og)
        
        epithelial_img_og_shape = epithelial_img_og.shape
        immune_img_og_shape = epithelial_img_og.shape

        # Register them
        transform_matrix = get_transformation_mat(ep_object_mask, immune_object_mask)
        sr = get_trans_mat_for_visualization(ep_object_mask, immune_object_mask)
        
        #output image to make sure transformation makes sense
        overlay_image_filename = os.path.join(overlap_folder, key)
        output_overlap_image(ep_object_mask, immune_object_mask, sr, overlay_image_filename)
       
       
        #apply registration matrix to dataframe
        #Subset caf_df_mov to the rows where filename equals the value
        subset_caf_df_mov = caf_df_mov[caf_df_mov['filename'] == value].copy()
        
        #read in an epithelial subset that matches
        
        # Extract the first description from caf_df_mov, to be able to find matching epithelial file
        first_description = subset_caf_df_mov['description'].iloc[0]
        # Specify the folder path
        folder_path = '000_Data/transformed_coor/ep_rot_pad_trans'
        # Find and read the matching CSV file into subset_ep_df_mov
        subset_ep_df_mov = find_and_read_csv(folder_path, first_description)
        
        # Apply rotation to subset_caf_df_mov
        subset_caf_df_mov = rotate_coordinates(subset_caf_df_mov, caf_keywords, immune_img_og_shape)

        # Apply rotation to subset_ep_df_mov
        subset_ep_df_mov = rotate_coordinates(subset_ep_df_mov, ep_keywords, epithelial_img_og_shape)

        #grab information necessary for padding correction
        smaller_height_img, height_difference, smaller_width_img, width_difference = compare_image_pair_sizes(epithelial_img_og, immune_img_og)
        
        #OK GREAT
        
        #now
        #if epithelial is smaller height, add half the height difference to the epithelial y value, and call it rot_pad
        if smaller_height_img == 'epithelial':
            subset_ep_df_mov['y_rot_pad'] = subset_ep_df_mov['y_rot_pad'] + height_difference / 2
            subset_caf_df_mov['y_rot_pad'] = subset_caf_df_mov['y_rot']
        else:
            subset_caf_df_mov['y_rot_pad'] = subset_caf_df_mov['y_rot'] + height_difference / 2
            subset_ep_df_mov['y_rot_pad'] = subset_ep_df_mov['y_rot_pad']
        if smaller_width_img == 'epithelial':
            subset_ep_df_mov['x_rot_pad'] = subset_ep_df_mov['x_rot_pad'] + height_difference / 2
            subset_caf_df_mov['x_rot_pad'] = subset_caf_df_mov['x_rot']
        else:
            subset_caf_df_mov['x_rot_pad'] = subset_caf_df_mov['x_rot'] + height_difference / 2
            subset_ep_df_mov['x_rot_pad'] = subset_ep_df_mov['x_rot_pad']

        # Apply the transformation matrix
        x_transformed, y_transformed = transform_coordinates(transform_matrix, subset_caf_df_mov['x_rot_pad'].values, subset_caf_df_mov['y_rot_pad'].values)

        subset_caf_df_mov['x_rot_pad_trans'] = x_transformed
        subset_caf_df_mov['y_rot_pad_trans'] = y_transformed
        
        
        x_transformed, y_transformed = transform_coordinates(transform_matrix, subset_ep_df_mov['x_rot_pad'].values, subset_ep_df_mov['y_rot_pad'].values)
        subset_ep_df_mov['x_rot_pad_trans_caf'] = x_transformed
        subset_ep_df_mov['y_rot_pad_trans_caf'] = y_transformed


        # Get the filename
        
        sample_name = subset_caf_df_mov['description'].iloc[0]
        csv_filename_caf = f"/Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/{sample_name}.csv"
        csv_filename_ep = f"/Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/ep_rot_pad_trans_2/{sample_name}.csv"
        
        # Write subset_df to CSV
        subset_caf_df_mov.to_csv(csv_filename_caf, index=False)
        subset_ep_df_mov.to_csv(csv_filename_ep, index=False)
        
        print(f"Transformed df has been written to {csv_filename}")

    except FileNotFoundError as e:
        # If a file is not found, print the error message and skip to the next iteration
        print(f"Skipping {key}, {value}: {e}")
        continue

    except Exception as e:
        # Handle any other exceptions to ensure the loop continues
        print(f"An error occurred with {key}, {value}: {e}")
        continue

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Skipping Ferris_TMA_2022_PDAC2_s0_a8_ac_full_Simple_Segmentation.tiff, PDAC2_s0_a18_ac_full_Simple Segmentation.tiff: File(s) not found: Ferris_TMA_2022_PDAC2_s0_a8_ac_full_Simple_Segmentation.tiff, PDAC2_s0_a18_ac_full_Simple Segmentation.tiff
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Skipping Ferris_TMA_2022_PDAC2_s0_a52_ac_full_Simple_Segmentation.tiff, PDAC2_s0_a39_ac_full_Simple Segmentation.tiff: File(s) not found: Ferris_TMA_2022_PDAC2_s0_a52_ac_full_Simple_Segmentation.tiff, PDAC2_s0_a39_ac_full_Simple Segmentation.tiff
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been writt

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Skipping Ferris_TMA_2022_PDAC2_s0_a100_ac_full_Simple_Segmentation.tiff, PDAC2_s0_a20_ac_full_Simple Segmentation.tiff: File(s) not found: Ferris_TMA_2022_PDAC2_s0_a100_ac_full_Simple_Segmentation.tiff, PDAC2_s0_a20_ac_full_Simple Segmentation.tiff
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been wri

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transf

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transf

...

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transf

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transf

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transf

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transf

Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transformed_coor/CAF_rot_pad_trans/PDAC2_Sector2Row2Column4_Punch1_Case146_ID_GSDM_2546.csv
Transformed df has been written to /Users/ferris/Desktop/Serial Section Overlay/000_Data/transf

# Visualization check

#next step:

Apply correction to immune data, THEN, load in the changed epithelial files, and apply another layer of correction based on caf data to most closely mimic how I modified the original images